In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import random

from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Importação dos dados
tabela = pd.read_csv('/content/order_reviews_dataset.csv')

In [ ]:
# Pré visualização - 5 linhas do topo
tabela.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [ ]:
# Verificação da quantidade de pedidos por score
np.unique(tabela.review_score, return_counts=True)

(array([1, 2, 3, 4, 5]), array([11858,  3235,  8287, 19200, 57420]))

In [ ]:
# Verificação da quantidade de pedidos por score, quando há uma avaliação
np.unique(tabela[tabela['review_comment_message'].fillna(0)!=0].review_score, return_counts=True)

(array([1, 2, 3, 4, 5]), array([ 9179,  2229,  3665,  6034, 20646]))

In [ ]:
# Filtro da tabela original para manter apenas as linhas com avaliações
tabela = tabela[tabela['review_comment_message'].fillna(0)!=0]

In [ ]:
# Importação stopwords e raízes para stemming
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [ ]:
# Função de limpeza do texto
def limpa_texto(texto):
  texto = texto.replace('á','a').replace('ã','a').replace('à','a').replace('é','e').replace('ê','e').replace('ô','o').replace('í','i').replace('ó','o').replace('ú','u').replace('ç','c').replace('\n','').replace('\r','')
  texto = re.sub('[^a-zA-Z0-9\s]','', texto)
  texto = re.sub(' +',' ', texto)
  return texto.lower()

In [ ]:
# Função de remoção de stopwords
def remove_stopwords(texto, lista_stopwords):
  return [palavra for palavra in texto.split(' ') if palavra not in lista_stopwords]

In [ ]:
# Função para stemming -- ajuste das palavras para manter apenas as raízes
def stem_palavras(lista_palavras):
  return [stemmer.stem(palavra) for palavra in lista_palavras]

In [ ]:
# Definição da lista de stopwords
stopwords = limpa_texto(' '.join(nltk.corpus.stopwords.words('portuguese'))).split(' ')

In [ ]:
# Criação do stemmer
stemmer = nltk.stem.RSLPStemmer()

In [ ]:
# Limpa palavras das avaliações
texto_limpo = limpa_texto(' '.join(tabela['review_comment_message'].values))

In [ ]:
# Remove stopwords
texto_limpo_s_stopwords = list(set(remove_stopwords(texto_limpo, stopwords)))

In [ ]:
# Aplica o stemming
texto_limpo_stemming = list(set(stem_palavras(texto_limpo_s_stopwords)))

In [ ]:
# Criação dos dicionários -- id > palavra e palavra > id
dicionario = dict((valor,chave) for chave,valor in enumerate(texto_limpo_stemming))
dicionario_inv = dict((chave,valor) for chave,valor in enumerate(texto_limpo_stemming))

In [ ]:
# Tokenização das frases -- leva 1min
frases_tokenizadas = []
for review in tabela['review_comment_message'].values:
  limpo = limpa_texto(review)
  remov = remove_stopwords(limpo, stopwords)
  stemm = stem_palavras([p for p in remov if p != ''])
  frases_tokenizadas.append([dicionario[palavra] for palavra in stemm])

In [ ]:
# Contagem das frequencias de palavras para remoção das palavras menos utilizadas
frequencias = pd.DataFrame(np.unique([token for sublista in frases_tokenizadas for token in sublista], return_counts=True)).T
frequencias.columns = ['token', 'freq']
frequencias['palavra'] = frequencias.token.apply(lambda x: dicionario_inv[x])

In [ ]:
# Define as 200 palavras mais utilizadas
lista_tokens_validos = frequencias.sort_values(by='freq', ascending=False).head(200).token.values

In [ ]:
# Adapta as frases para o formato bag of words
frases_bow = [[1 if token_valido in frase else 0 for token_valido in lista_tokens_validos] for frase in frases_tokenizadas]

Preparação dos dados

In [ ]:
# Ajusta as categorias -- notas 4 e 5 = categoria 1 / notas 1, 2 e 3 = categoria 0
categorias = tabela['review_score'].apply(lambda x: 1 if x>=4 else 0).values

In [ ]:
# Criação do database que será utilizado na modelagem
db = pd.DataFrame(frases_bow, columns=lista_tokens_validos)
db['categoria'] = categorias

In [ ]:
# Pré visualização dos dados para modelagem
db.head()

,13876,11413,9379,13709,2895,5356,13489,11543,1281,4057,...,13065,8205,7000,3211,7407,7168,7763,11428,9372,categoria
0,0,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Balanceamento das categorias
db_cat_0 = db[db['categoria']==0]
db_cat_1 = pd.DataFrame(random.sample(list(db[db['categoria']==1].values), len(db_cat_0)), columns=db.columns)

db_ = pd.concat([db_cat_0, db_cat_1], axis=0)

In [ ]:
# Divisão dos dados entre treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(db_.drop('categoria', axis=1), db_.categoria, train_size=.7, stratify=db_.categoria)

# Modelagem

**Para os testes com as redes densas e recorrentes, utilize as variáveis `X_train` e `X_test` para as features.**

## Baseline

In [ ]:
# Definição de um modelo AdaBoost para classificação, baseado em 100 estimadores
modelo = AdaBoostClassifier(n_estimators=100).fit(X_train, y_train)

In [ ]:
# Avaliação do modelo - dados de treinamento
modelo.score(X_train, y_train)

0.7926262913467917

In [ ]:
# Avaliação do modelo - dados de teste
modelo.score(X_train, y_train)

0.7926262913467917

## Redes Densas

## Redes Recorrentes